In [1]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertConfig, BertForMaskedLM

In [3]:
config = BertConfig(
    vocab_size=40000,
    hidden_size=256,
    num_hidden_layer=4,
    num_attention_heads=4,
    intermediate_size=1024,
    max_position_embeddings=1024
)

In [4]:
model = BertForMaskedLM(config) #새 모델을 만든것임 우리가 하고 싶은 것은 파인튜닝.
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(40000, 256, padding_idx=0)
      (position_embeddings): Embedding(1024, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=T

config를 들고 오고 싶다면...

In [5]:
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertForMaskedLM

In [6]:
config = BertConfig.from_pretrained("bert-base-uncased") #weight를 들고 오지는 않음.
model = BertForMaskedLM(config) #config만 가져온것임. 가중치는 랜덤함. 우리가 원하는 것은 사전 학습이다.
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [7]:
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertForMaskedLM

In [8]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased") #weight를 다 가져옴. 마지막 까지 다 학습이 되어있다. #근데 토큰아이저를 안가져왔다.

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from transformers import BertTokenizerFast #모델기반 토크나이저를 써야된다. pre-train 모델 자체가 특정 토크나이저로 학습했기 때문이다.

from pprint import pprint

In [10]:
tokenizer=BertTokenizerFast.from_pretrained("bert-base-uncased")
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [11]:
text =[ "Hello my name is DawonLee","I go to school"]
pprint(tokenizer(text))

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 7592, 2026, 2171, 2003, 4830, 19291, 10559, 102],
               [101, 1045, 2175, 2000, 2082, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]}


In [12]:
pprint(tokenizer(text,return_tensors="pt",padding=True))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0]]),
 'input_ids': tensor([[  101,  7592,  2026,  2171,  2003,  4830, 19291, 10559,   102],
        [  101,  1045,  2175,  2000,  2082,   102,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [13]:
encodings = tokenizer(text,return_tensors="pt",padding=True)
pprint(tokenizer.convert_ids_to_tokens(encodings["input_ids"][0]))

['[CLS]', 'hello', 'my', 'name', 'is', 'da', '##won', '##lee', '[SEP]']


In [14]:
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast

In [15]:
encodings = tokenizer(text,return_tensors="pt",padding=True,return_token_type_ids=False)
pprint(tokenizer.convert_ids_to_tokens(encodings["input_ids"][1]))

['[CLS]', 'i', 'go', 'to', 'school', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [16]:
encodings = tokenizer(
    "We are very happy to [MASK] you the Transformers library.",return_tensors="pt"
)
pprint(encodings)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,   103,  2017,  1996, 19081,
          3075,  1012,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [17]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
outputs = model(**encodings)
pprint(outputs) #이 로짓은 bert 마지막 batchsize*seqlen*vocab_size 이것의 확률이다.
print(outputs["logits"].shape)

{'logits': tensor([[[ -6.7982,  -6.7357,  -6.7167,  ...,  -5.9878,  -5.8950,  -4.0933],
         [-12.1013, -11.6499, -11.9993,  ...,  -9.1666,  -8.3991, -11.8748],
         [-12.7212, -12.4337, -12.6175,  ...,  -9.1279,  -9.4953,  -9.6976],
         ...,
         [ -6.5864,  -6.6156,  -6.4550,  ...,  -7.0086,  -6.3760,  -6.2223],
         [-13.0880, -12.9470, -12.9260,  ..., -10.1065, -10.3579,  -8.8531],
         [-11.6692, -12.0971, -11.7947,  ...,  -9.0227,  -9.5557,  -7.3882]]],
       grad_fn=<ViewBackward0>)}
torch.Size([1, 13, 30522])


In [19]:
print(outputs.logits.argmax(dim=-1))

tensor([[ 1012,  2057,  2024,  2200,  7537,  2000,  2265,  2017,  1996, 19081,
          3075,  1012,  1012]])


In [20]:
print(tokenizer.decode(outputs.logits.argmax(dim=-1).squeeze(0)))

. we are very pleased to show you the transformers library..


In [21]:
print(tokenizer.decode(outputs.logits.argmax(dim=-1)[0]))

. we are very pleased to show you the transformers library..


In [22]:
from transformers import BertForSequenceClassification

In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from datasets import load_dataset
data=load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
import re

In [29]:
def preprocess(sample):
    return {
        "text":" ".join(re.sub(r"<[^(?:/>)]+/>",' ',sample["text"]).split()),
        "label":sample["label"]
    }
preprocessed = data.map(preprocess)

  0%|          | 0/25000 [00:00<?, ?ex/s]

  0%|          | 0/25000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

In [30]:
from transformers import BertTokenizerFast,AutoTokenizer

In [31]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased',use_fast=True)

In [34]:
preprocessed = preprocessed.map(
    lambda sample: tokenizer(sample['text'],truncation=True),
    remove_columns = ['text'],
    batched=True
)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [35]:
from transformers import DataCollatorWithPadding


In [36]:
collator = DataCollatorWithPadding(tokenizer)

In [37]:
collator(preprocessed['train'][:5])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  1045, 12524,  ...,     0,     0,     0],
        [  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  2065,  2069,  ...,     0,     0,     0],
        [  101,  2023,  2143,  ...,     0,     0,     0],
        [  101,  2821,  1010,  ...,  2007,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 0, 0, 0, 0])}

In [39]:
from torch.utils.data import DataLoader
train_loader = DataLoader(preprocessed['train'],batch_size=16,collate_fn=collator,shuffle=True)

In [38]:
from transformers import BertForSequenceClassification,AutoModelForSequenceClassification

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [43]:
import torch
optimizer = torch.optim.AdamW([
    {"params":model.bert.parameters(),'lr':3e-5},
    {"params":model.classifier.parameters(),'lr':1e-3}
    ])

In [44]:
model.train()
for epoch in range(3):
    print(f"Epoch: {epoch}")
    for encodings in train_loader:
        encodings = {key:value.cuda() for key,value in encodings.items()}
        outputs = model(**encodings)
        outputs.loss.backward()
        print("\rLoss: ",outputs.loss.item(),end=" ")
        optimizer.step()
        optimizer.zero_grad(set_to_none=False)

Epoch: 0
Loss:  0.15717372298240662 Epoch: 1
Loss:  0.03633458539843559 

KeyboardInterrupt: ignored

preprocessed 까지 한다.

In [45]:
!pip install datasets
from datasets import load_dataset
data=load_dataset("imdb")
import re
def preprocess(sample):
    return {
        "text":" ".join(re.sub(r"<[^(?:/>)]+/>",' ',sample["text"]).split()),
        "label":sample["label"]
    }
preprocessed = data.map(preprocess)

  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    num_train_epochs = 3.0,
    per_device_train_batch_size=1e
    output_dir="dump/test",

)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = preproccessed["train"],
    eval_dataset = preprocessed["test"],
    data_collator=collator
)

SyntaxError: ignored

In [ ]:
trainer.train()